In [ ]:
import sys
import pathlib
module_path = str(pathlib.Path().absolute()).replace('/notebooks', '')
sys.path.append(module_path)

In [ ]:
from scripts.jobs.dataset_creation import ReadRasterFile, StridedArrayGenerator, PreProcessBorderRaster, StoreCompositeDataHDF5, PreProcessPopulationRaster

raster_dem_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/dem_aggr_rst.tif'
raster_pop_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'
raster_nuts_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/nuts_rst.tif'

model_name = 'pop_nuts'

rasters = {
    'pop': {
        'type': 'input',
        'data': PreProcessPopulationRaster(ReadRasterFile(raster_pop_fn)),
        'bad_value_threshold': -1000
    },
    'nuts': {
        'type': 'output',
        'data': PreProcessBorderRaster(ReadRasterFile(raster_nuts_fn), bad_value=-1),
        'bad_value_threshold': -1
    }
}
strided_generator = StridedArrayGenerator(rasters, window_size = 100)
step = 0
while True:
    (train_x, train_y), (test_x, test_y) = next(strided_generator)
    if train_x.shape[0] == 0:
        break
    else:
        for i in range(2):
            selection_mask = [(x == i)[0] for x in train_y]
            StoreCompositeDataHDF5(train_x[selection_mask], train_y[selection_mask], model_name + '_train_' + str(i) + '_' + str(step))
            selection_mask = [(x == i)[0] for x in test_y]
            StoreCompositeDataHDF5(test_x[selection_mask], test_y[selection_mask], model_name + '_test_' + str(i) + '_' + str(step))
        step += 1


In [ ]:

from scripts.jobs.dataset_creation import CreateTFDatasetFromCompositeGenerator
from scripts.jobs.networks.conv_classifier import TrainConvClassifierModel

model_name = 'pop_nuts'

train_dataset = CreateTFDatasetFromCompositeGenerator(model_name + '_train', 2, batch_size = 8, window_size = 100)
test_dataset = CreateTFDatasetFromCompositeGenerator(model_name + '_test', 2, batch_size = 8, window_size = 100)

model, history = TrainConvClassifierModel(train_dataset, test_dataset, num_epochs = 10, steps_per_epoch = 1000)  # int(round(500000 / 64, 0)))
model.save('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/' + model_name + '_model') 


In [1]:
from scripts.jobs.dataset_creation import CreateTFDatasetFromInMemoryGenerator, InMemoryStridedArrayGenerator
from scripts.jobs.networks.conv_classifier import TrainConvClassifierModel
from scripts.jobs.dataset_creation import ReadRasterFile, PreProcessBorderRaster, PreProcessPopulationRaster, PreProcessLogarithmPopulationRaster,PreprocessForResnet, PreProcessWWRaster
from scripts.jobs.networks.siamese import resnet, simple_resnet

raster_ww_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/ww_aggr_rst.tif'
raster_dem_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/dem_aggr_rst.tif'
raster_pop_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'
raster_nuts_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/nuts_rst.tif'

model_name = 'dem_pop_ww_nuts'

rasters = {
    'dem': {
        'type': 'input',
        'data': PreprocessForResnet(ReadRasterFile(raster_dem_fn)),
        'bad_value_threshold': -1000
    },
    'pop': {
        'type': 'input',
        'data': PreprocessForResnet(PreProcessLogarithmPopulationRaster(PreProcessPopulationRaster(ReadRasterFile(raster_pop_fn)))),
        'bad_value_threshold': -1000
    },
    'ww': {
        'type': 'input',
        'data': PreprocessForResnet(PreProcessWWRaster(ReadRasterFile(raster_ww_fn))),
        'bad_value_threshold': -1000
    },
    'nuts': {
        'type': 'output',
        'data': PreProcessBorderRaster(ReadRasterFile(raster_nuts_fn), bad_value=-1),
        'bad_value_threshold': -1
    }
}
gen = InMemoryStridedArrayGenerator(rasters, window_size = 100)
train_dataset = CreateTFDatasetFromInMemoryGenerator(gen, batch_size = 64, window_size = 100, channel_n = 3)
# internal_model = resnet(['pop'], hidden_layer_size = 100, input_size = 100)
internal_model = simple_resnet(input_size = 100, channel_n = 3)
internal_model = None

model, history = TrainConvClassifierModel(
    train_dataset,
    train_dataset,
    num_epochs = 30,
    steps_per_epoch = 10000,
    internal_model = internal_model
)
model.save('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/' + model_name + '_model') 

In [1]:
from scripts.jobs.dataset_creation import CreateTFDatasetFromInMemoryGenerator, InMemoryStridedArrayGenerator, create_generator_sequence
from scripts.jobs.networks.conv_classifier import TrainConvClassifierModel
import scripts.jobs.dataset_creation as dataset_creation
from scripts.jobs.networks.siamese import resnet, simple_resnet, simple_vgg

raster_ww_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/ww_aggr_rst.tif'
raster_dem_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/dem_aggr_rst.tif'
raster_pop_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'
raster_nuts_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/nuts_rst.tif'

model_name = 'dem_pop_ww_nuts'
window_size = 100

rasters = {
    'pop': {
        'type': 'input',
        'data': dataset_creation.PreprocessForResnet(dataset_creation.PreProcessLogarithmPopulationRaster(dataset_creation.PreProcessPopulationRaster(dataset_creation.ReadRasterFile(raster_pop_fn)))),
        'bad_value_threshold': -1000
    },
    'dem': {
        'type': 'input',
        'data': dataset_creation.PreprocessForResnet(dataset_creation.PreprocessDEMRaster(dataset_creation.ReadRasterFile(raster_dem_fn))),
        'bad_value_threshold': -1000
    },
    'ww': {
        'type': 'input',
        'data': dataset_creation.PreprocessForResnet(dataset_creation.PreProcessWWRaster(dataset_creation.ReadRasterFile(raster_ww_fn))),
        'bad_value_threshold': -1000
    },
    'nuts': {
        'type': 'output',
        'data': dataset_creation.PreProcessBorderRaster(dataset_creation.ReadRasterFile(raster_nuts_fn), bad_value=-1),
        'bad_value_threshold': -1
    }
}

generator_sequence = create_generator_sequence(rasters, window_size = window_size)
train_size = int(len(generator_sequence) * 0.8)

train_gen = InMemoryStridedArrayGenerator(rasters, window_size = window_size, generator_sequence = generator_sequence[0:5000])
train_dataset = CreateTFDatasetFromInMemoryGenerator(train_gen, batch_size = 64, window_size = window_size, channel_n = 3)
test_gen = InMemoryStridedArrayGenerator(rasters, window_size = window_size, generator_sequence = generator_sequence[train_size:])
test_dataset = CreateTFDatasetFromInMemoryGenerator(test_gen, batch_size = 64, window_size = window_size, channel_n = 3)
# internal_model = resnet(['pop'], hidden_layer_size = 100, input_size = 100)
# internal_model = simple_resnet(input_size = window_size, channel_n = 3)
internal_model = simple_vgg(input_size = window_size, channel_n = 3)

model, history = TrainConvClassifierModel(
    train_dataset,
    test_dataset,
    num_epochs = 10,
    steps_per_epoch = 1000,
    internal_model = internal_model
)

model.save('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/' + model_name + '_model') 

AttributeError: module 'scripts.jobs.dataset_creation' has no attribute 'PreProcessDEMRaster'

In [ ]:
import tensorflow as tf
import numpy as np
from scripts.jobs.dataset_creation import ReadRasterFile, PreProcessPopulationRaster, PreProcessLogarithmPopulationRaster, PreprocessForResnet
from scripts.jobs.model_prediction import PredictClassifierRaster, WriteResultRaster

model_name = 'dem_pop_ww_nuts'

raster_pop_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'
model = tf.keras.models.load_model('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/' + model_name + '_model') 

a = PreprocessForResnet(PreProcessLogarithmPopulationRaster(PreProcessPopulationRaster(ReadRasterFile(raster_pop_fn))))
a = np.stack([a] * 3, axis = -1)

data = PredictClassifierRaster(a, model, stride = 33, channel_n = 3)

out_rst_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/result_pop_nuts.tif'
WriteResultRaster(data, raster_fn, out_rst_fn, channels=0)

In [ ]:
from scripts.jobs.dataset_creation import ReadRasterFile, PreProcessBorderRaster, StoreCompositeDataHDF5, PreProcessPopulationRaster, PreprocessForResnet, PreProcessLogarithmPopulationRaster, PreProcessDEMRaster
from matplotlib import pyplot

raster_nuts_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/nuts_rst.tif'
raster_dem_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/dem_aggr_rst.tif'
raster_pop_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'

pyplot.figure(figsize = (10,10))
a = PreProcessBorderRaster(ReadRasterFile(raster_nuts_fn), bad_value = -1)
pyplot.imshow(a, cmap='tab10')
pyplot.show()

pyplot.figure(figsize = (10,10))
pyplot.imshow(PreprocessForResnet(PreProcessLogarithmPopulationRaster(PreProcessPopulationRaster(ReadRasterFile(raster_pop_fn)))), cmap='viridis')  # viridis
pyplot.show()
pyplot.figure(figsize = (10,10))
a = PreprocessForResnet(PreProcessDEMRaster(ReadRasterFile(raster_dem_fn)))
pyplot.imshow(a, cmap='viridis')
pyplot.show()


In [ ]:
a = ReadRasterFile('/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif')
a[a < -10] = -10
a[(a < 3) & (a > -10)] = 0
a[a > 2] = 1
pyplot.hist(a, bins='auto') 
pyplot.show()